In [1]:
# deletes engine - run when you have to restart
!rm world_dev.sqlite

rm: world_dev.sqlite: No such file or directory


In [2]:
# sqlalchemy dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import func

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text, Numeric

import os

In [3]:
# pandas dependencies
import pandas as pd
import pandas_profiling

# display multiple print results on one line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# flask 
from flask import Flask, jsonify

In [5]:
# read csv
df = pd.read_csv('data.csv')
updated_df = df.rename(columns={"Unnamed: 0": "id",
                  "Country Name": "CountryName",
                  "Gini Coefficient": "GiniCoefficient",
                  "GDP PPP": "GDPPPP",
                  "Homicide per 100k": "Homicideper100k",
                  "Mean": "GDPMean"})
updated_df.set_index("id").head()
updated_df.shape

,CountryName,Region,IncomeGroup,GiniCoefficient,GDPPPP,Homicideper100k,HDI,GDPMean
id,,,,,,,,
0,Afghanistan,South Asia,Low income,0.0,1926.357336,0.0,0.479,5.607081
1,Angola,Sub-Saharan Africa,Lower middle income,0.0,6635.242750,9.6,0.533,3.777340
2,Albania,Europe & Central Asia,Upper middle income,0.0,11282.793580,2.3,0.764,2.293126
3,United Arab Emirates,Middle East & North Africa,High income,0.0,70254.831570,0.7,0.840,4.148458
4,Argentina,Latin America & Caribbean,Upper middle income,0.0,20340.292420,6.5,0.827,2.199652


(139, 9)

In [6]:
# pandas profile report
# pandas_profiling.ProfileReport(updated_df)

In [7]:
# create sqlite database
engine = create_engine("sqlite:///world_dev.sqlite")
conn = engine.connect()

In [8]:
# declarative_base
# Base = declarative_base()
Base = automap_base()

In [9]:
# create world_dev metadata class
class world_dev(Base):
    __tablename__ = 'worldDev'
    
    id = Column(Integer, primary_key=True)
    CountryName = Column(Text(64))
    Region = Column(Text(64))
    IncomeGroup = Column(Text(64))
    GiniCoefficient = Column(Float(64))
    GDPPP = Column(Float(64))
    Homicideper100k = Column(Float(64))
    HDI = Column(Float(64))
    GDPMean = Column(Float(64))   
    
    def __repr__(self):
        return f"id={self.ID}"

In [10]:
# create a "metadata" layer that abstracts our SQL Database
Base.metadata.create_all(engine)
engine.table_names()

['worldDev']

In [11]:
# create a session object to connect to DB
session = Session(bind=engine)

In [12]:
# add updated_df csv data
updated_df.to_sql('world_dev', conn, if_exists='append', index=False)

In [13]:
# read back world_dev class to see if updated_df data was appended to world_dev.sqlite
data_world_dev = pd.read_sql("SELECT * FROM world_dev", conn)
# data_world_dev.head()
# data_world_dev.tail()
data_world_dev.shape

(139, 9)

In [14]:
# store data_world_dev into new variable and set index to id
final_sql_df = data_world_dev.set_index('id')
final_sql_df.head()

,CountryName,Region,IncomeGroup,GiniCoefficient,GDPPPP,Homicideper100k,HDI,GDPMean
id,,,,,,,,
0,Afghanistan,South Asia,Low income,0.0,1926.357336,0.0,0.479,5.607081
1,Angola,Sub-Saharan Africa,Lower middle income,0.0,6635.242750,9.6,0.533,3.777340
2,Albania,Europe & Central Asia,Upper middle income,0.0,11282.793580,2.3,0.764,2.293126
3,United Arab Emirates,Middle East & North Africa,High income,0.0,70254.831570,0.7,0.840,4.148458
4,Argentina,Latin America & Caribbean,Upper middle income,0.0,20340.292420,6.5,0.827,2.199652


In [15]:
# save to csv
# final_sql_df.to_csv('worldDev.csv')

In [16]:
# create object for worldDev.csv
worldDevcsv = 'worldDev.csv'

In [17]:
# function to populate table
def populate_table (engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe
    insert_df = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = insert_df.to_dict(orient='records')
    print(data[:1])
    
    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)

# Call the function to insert the data for each table
# world_dev is the class
populate_table(engine, world_dev.__table__, worldDevcsv)

[{'id': 0, 'CountryName': 'Afghanistan', 'Region': 'South Asia', 'IncomeGroup': 'Low income', 'GiniCoefficient': 0.0, 'GDPPPP': 1926.357336, 'Homicideper100k': 0.0, 'HDI': 0.479, 'GDPMean': 5.60708087}]


In [18]:
# check to see if data was added to table
engine.execute("SELECT * FROM worldDev LIMIT 1").fetchall()

[(0, 'Afghanistan', 'South Asia', 'Low income', 0.0, None, 0.0, 0.479, 5.60708087)]

### End Database Engineering Section
<br>
<hr>

### Reflect Table Section
<hr>

In [19]:
# Use SQLAlchemy automap_base() to refelct tables into classes and save a reference to those classes
Base = automap_base()

In [20]:
# use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [21]:
# print all of the classes mapped to the Base
Base.classes.keys()

['worldDev']

In [22]:
# assign worldDev class to variables
WorldDev = Base.classes.worldDev

In [23]:
# create a session
session = Session(engine)

In [24]:
# Display the row's columns and data in dict format
results = session.query(WorldDev.CountryName, WorldDev.GiniCoefficient).all()

In [25]:
# Prints Country Name and GiniCoefficient
results

[('Afghanistan', 0.0),
 ('Angola', 0.0),
 ('Albania', 0.0),
 ('United Arab Emirates', 0.0),
 ('Argentina', 0.0),
 ('Armenia', 32.4),
 ('Australia', 0.0),
 ('Austria', 0.0),
 ('Azerbaijan', 0.0),
 ('Burundi', 0.0),
 ('Belgium', 0.0),
 ('Benin', 47.8),
 ('Burkina Faso', 0.0),
 ('Bangladesh', 0.0),
 ('Bulgaria', 0.0),
 ('Bosnia and Herzegovina', 0.0),
 ('Belarus', 26.7),
 ('Belize', 0.0),
 ('Brazil', 51.3),
 ('Bhutan', 0.0),
 ('Botswana', 0.0),
 ('Central African Republic', 0.0),
 ('Canada', 0.0),
 ('Switzerland', 0.0),
 ('Chile', 47.7),
 ('China', 0.0),
 ('Cameroon', 0.0),
 ('Colombia', 51.1),
 ('Costa Rica', 48.2),
 ('Cuba', 0.0),
 ('Cyprus', 0.0),
 ('Czech Republic', 0.0),
 ('Germany', 0.0),
 ('Djibouti', 0.0),
 ('Denmark', 0.0),
 ('Dominican Republic', 44.9),
 ('Algeria', 0.0),
 ('Ecuador', 46.5),
 ('Eritrea', 0.0),
 ('Spain', 0.0),
 ('Estonia', 0.0),
 ('Ethiopia', 0.0),
 ('Finland', 0.0),
 ('Fiji', 0.0),
 ('France', 0.0),
 ('Gabon', 0.0),
 ('United Kingdom', 0.0),
 ('Georgia', 38.5),